# Quantum Neural Network with Flask API
This notebook demonstrates how to set up a Flask web API for predicting stock prices using a pre-trained quantum neural network (QNN). The QNN is created using Qiskit and PyTorch, and the API offers two routes: one for predicting stock prices and another for fetching stock data from Yahoo Finance.

## 1. Import Necessary Libraries
We begin by importing the necessary libraries for building the Flask API, processing data, and using quantum computing techniques for stock price prediction. This includes libraries for web development, quantum computing, data manipulation, and stock data fetching.

In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import torch
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import TwoLayerQNN
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import ParameterVector
import yfinance as yf
import os


## 2. Set Up Flask App
In this section, we create the Flask app that will serve as the API for making predictions and fetching stock data. First, we initialize the Flask app.

In [ ]:
# Set up the Flask app
app = Flask(__name__)


## 3. Create Quantum Neural Network (QNN)
Now we define the function to create the quantum neural network. This will set up the feature map and ansatz using Qiskit's `RealAmplitudes` circuit for quantum feature extraction.

In [ ]:
# Create a TwoLayerQNN model
def create_qnn(num_qubits):
    """Create a TwoLayerQNN with feature map and ansatz."""
    feature_map = RealAmplitudes(num_qubits, reps=1)
    feature_map_params = ParameterVector('fm_theta', feature_map.num_parameters)
    feature_map.assign_parameters(feature_map_params, inplace=True)

    ansatz = RealAmplitudes(num_qubits, reps=1)
    ansatz_params = ParameterVector('ansatz_theta', ansatz.num_parameters)
    ansatz.assign_parameters(ansatz_params, inplace=True)

    qnn = TwoLayerQNN(
        num_qubits=num_qubits,
        feature_map=feature_map,
        ansatz=ansatz,
        quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator'))
    )
    return qnn


## 4. Integrate QNN with PyTorch
We now integrate the quantum neural network with PyTorch. The QNN will be wrapped using the `TorchConnector`, and we will add a simple linear layer to the model.

In [ ]:
# Integrate QNN with PyTorch
qnn = create_qnn(num_qubits=4)
model = TorchConnector(qnn)
model = torch.nn.Sequential(model, torch.nn.Linear(1, 1))  # Add a linear layer for output

## 5. Load the Pre-trained Model
In this section, we attempt to load the pre-trained model parameters. If the model file exists, we load the state dictionary; otherwise, an error message is displayed.

In [ ]:
# Load the trained model parameters from the specified file
MODEL_PATH = "../model/quantum_nn_model.pth"
if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH))
    print("Model loaded successfully.")
else:
    print("Model file not found. Ensure the model is trained and saved.")

## 6. Define Prediction Route
We now define the route for predicting stock prices. The route accepts a file upload containing stock data, processes it, and uses the QNN model to make predictions.

In [ ]:
# Flask route for predicting stock prices
@app.route('/predict', methods=['POST'])
def predict():
    """Predict stock prices using the pre-trained QNN model."""
    print('predicting')
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file uploaded."}), 400

        # Read the uploaded CSV file containing stock data
        file = request.files['file']
        df = pd.read_csv(file)

        # Preprocess the input data by selecting relevant features
        features = ['open', 'high', 'low', 'close', 'volume', 'daily_return', '5_day_moving_avg', '30_day_moving_avg']
        X = df[features].values

        # Convert features to PyTorch tensor for prediction
        X_torch = torch.tensor(X, dtype=torch.float32)

        # Make predictions using the model
        predictions = model(X_torch).detach().numpy()
        df['predictions'] = predictions

        # Return the predictions as a JSON response
        return df.to_json(orient='records')
    except Exception as e:
        return jsonify({"error": str(e)}), 500

## 7. Define Stock Data Fetching Route
This route fetches stock data from Yahoo Finance, processes it by calculating daily returns and moving averages, and returns the data as a JSON response.

In [ ]:
# Flask route for fetching stock data from Yahoo Finance
@app.route('/fetch_stock', methods=['GET'])
def fetch_stock():
    """Fetch stock data from Yahoo Finance."""
    try:
        ticker = request.args.get('ticker', default=None, type=str)
        if not ticker:
            return jsonify({"error": "Ticker symbol is required."}), 400

        # Fetch stock data for the specified ticker
        data = yf.download(ticker, period="1y", interval="1d")
        data.reset_index(inplace=True)
        data['daily_return'] = data['Adj Close'].pct_change()
        data['5_day_moving_avg'] = data['Adj Close'].rolling(window=5).mean()
        data['30_day_moving_avg'] = data['Adj Close'].rolling(window=30).mean()

        # Return the processed stock data as a JSON response
        return data.to_json(orient='records')
    except Exception as e:
        return jsonify({"error": str(e)}), 500

## 8. Run the Flask Application
Finally, we run the Flask application, which will allow us to interact with the API for making stock price predictions and fetching stock data.

In [ ]:
# Run the Flask app in debug mode
if __name__ == '__main__':
    app.run(debug=True)